In [ ]:
!pip install tensorflow-gpu-1.15.5
!pip install git+https://github.com/openai/baselines.git@ea25b9e8b234e6ee1bca43083f8f3cf974143998

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu-1.15.5 (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu-1.15.5
  Cloning https://github.com/openai/baselines.git (to revision ea25b9e8b234e6ee1bca43083f8f3cf974143998) to /tmp/pip-req-build-rlc_k_ry
  Running command git clone -q https://github.com/openai/baselines.git /tmp/pip-req-build-rlc_k_ry
     |████████████████████████████████| 1.6MB 8.6MB/s 
  Created wheel for baselines: filename=baselines-0.1.6-cp37-none-any.whl size=220681 sha256=097c0786d768df857ab9de7f1afd55a52afeaff6cc1a37f5c46586759ee2209b
  Stored in directory: /tmp/pip-ephem-wheel-cache-hyy9u0xb/wheels/d1/e9/68/924a24fbf16f3feb881d97210a6379eeb41c0fd6f930112a39
  Created wheel for gym: filename=gym-0.15.7-cp37-none-any.whl size=1648842 sha256=aa54daef1aff968f45988802729d6effa134349e5b789b4f870ab45dd02ec68e
  Stored in directory: /root/.cache/pip/wheels/35/60/6a/f9c27ae133abaf5a5687ed2fa8ed19627d7fac5d843a27572b


In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
%tensorflow_version 1.x
import time
import gym
import random
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os

from baselines.ppo2 import ppo2
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

#from stable_baselines import DQN
from baselines import deepq
#from stable_baselines import deepq
from baselines import bench
from baselines import logger
import tensorflow as tf

from baselines.common.tf_util import make_session

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train_one_hot = keras.utils.to_categorical(y_train, num_classes)
y_test_one_hot = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
class MnistEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        super().__init__()

        self.action_space = gym.spaces.Discrete(10)
        self.observation_space = gym.spaces.Box(low=0, high=1,
                                                shape=(28, 28, 1),
                                                dtype=np.float32)

        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0

    def step(self, action):
        done = False
        reward = int(action == self.expected_action)

        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {}

    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y[next_obs_idx])
            obs = self.x[next_obs_idx]

        else:
            obs = self.x[self.dataset_idx]
            self.expected_action = int(self.y[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()

        return obs



In [ ]:
from sklearn.model_selection import RandomizedSearchCV


Using TensorFlow backend.


In [ ]:
def mnist_dqn():
    logger.configure(dir='./logs/mnist_dqn', format_strs=['stdout', 'tensorboard'])
    env = MnistEnv(images_per_episode=1)
    env = bench.Monitor(env, logger.get_dir())

    model = deepq.learn(
        env,
        "mlp",
        num_layers=1,
        num_hidden=64,
        activation=tf.nn.relu,
        hiddens=[32],
        dueling=True,
        lr=1e-6,
        total_timesteps=int(1.2e5),
        buffer_size=10000,
        
        exploration_fraction=0.1,
        exploration_final_eps=0.01,
        train_freq=5,
        learning_starts=10000,
        target_network_update_freq=1000,
    )
   
    model.save('dqn_mnist.pkl')
    env.close()
    
    return model

start_time = time.time()
dqn_model = mnist_dqn()
print("DQN Training Time:", time.time() - start_time)

Logging to ./logs/mnist_dqn





/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Streaming output truncated to the last 5000 lines.
| episodes                | 3.68e+04 |
| mean 100 episode reward | 0.1      |
| steps                   | 3.68e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 1        |
| episodes                | 3.69e+04 |
| mean 100 episode reward | 0.1      |
| steps                   | 3.69e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 1        |
| episodes                | 3.7e+04  |
| mean 100 episode reward | 0.2      |
| steps                   | 3.7e+04  |
--------------------------------------
--------------------------------------
| % time spent exploring  | 1        |
| episodes                | 3.71e+04 |
| mean 100 episode reward | 0.2      |
| steps                   | 3.71e+04 |
--------------------------------------
--------------------------------------
| % time spent exploring  | 1        |
| episodes   

In [ ]:
def mnist_dqn_eval(dqn_model):
    attempts, correct = 0,0

    env = MnistEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, _ = env.step(dqn_model(obs[None])[0])

                attempts += 1
                if rew > 0:
                    correct += 1

    except StopIteration:
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format((float(correct) / attempts) * 100))

mnist_dqn_eval(dqn_model)

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 40.06801360272054%
